In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3

In [4]:
datagen = ImageDataGenerator(
                rescale=1./255,
                horizontal_flip=True,
                rotation_range=0.2)

In [6]:
train_generator = datagen.flow_from_directory(
        'dataset/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse")

Found 1506 images belonging to 3 classes.


In [7]:
val_generator = datagen.flow_from_directory(
        'dataset/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse")

Found 215 images belonging to 3 classes.


In [8]:
test_generator = datagen.flow_from_directory(
        'dataset/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse")

Found 431 images belonging to 3 classes.


In [9]:
class_names = list(train_generator.class_indices.keys())
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [10]:
model = tf.keras.models.load_model("last_model.h5")

In [11]:
scores = model.evaluate(test_generator)
scores

14/14 [==============================] - 23s 1s/step - loss: 0.1561 - accuracy: 0.9606


[0.15607872605323792, 0.9605568647384644]

## 1) Post Training Quantization

#### Let's First try TFLite conversion without Quantization

In [47]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_models/1")
tflite_model = converter.convert()

In [48]:
len(tflite_model)

741368

#### Now let's apply  Quantization before converting to TFLite model 

In [46]:
converter2 = tf.lite.TFLiteConverter.from_saved_model("./saved_models/1")
converter2.optimazations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter2.convert()

In [49]:
len(tflite_quant_model)

741368

In [57]:
with open("tflite_quant_model.tflite", "wb") as f:
    f.write(tflite_quant_model)

## 2) Quantization aware Training

In [50]:
import tensorflow_model_optimization as tfmop
quantize_model = tfmop.quantization.keras.quantize_model
q_aware_model = quantize_model(model)

In [51]:
q_aware_model.compile("adam",
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                     metrics=["accuracy"])

In [52]:
q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 256, 256, 3)       3         
_________________________________________________________________
quant_conv2d (QuantizeWrappe (None, 254, 254, 32)      963       
_________________________________________________________________
quant_max_pooling2d (Quantiz (None, 127, 127, 32)      1         
_________________________________________________________________
quant_conv2d_1 (QuantizeWrap (None, 125, 125, 64)      18627     
_________________________________________________________________
quant_max_pooling2d_1 (Quant (None, 62, 62, 64)        1         
_________________________________________________________________
quant_conv2d_2 (QuantizeWrap (None, 60, 60, 64)        37059     
_________________________________________________________________
quant_max_pooling2d_2 (Quant (None, 30, 30, 64)        1

In [53]:
q_aware_model.fit(train_generator, 
                 epochs=1)

48/48 [==============================] - 174s 4s/step - loss: 0.0401 - accuracy: 0.9821


In [54]:
scores = q_aware_model.evaluate(test_generator)
scores

14/14 [==============================] - 17s 1s/step - loss: 0.0454 - accuracy: 0.9838


[0.04538000747561455, 0.9837586879730225]

In [60]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimazations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_aware_quant_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmp4gu6tw4z\assets


INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmp4gu6tw4z\assets


In [61]:
len(tflite_aware_quant_model)

1499312